In [145]:
import tkinter as tk
from tkinter import messagebox
import random

class AI:
    
    def __init__(self,board,blackpos):
        self.board=board
        self.blackpos=blackpos
        self.all_plays={}
        self.aiplay()
    
    def is_white(self,piece):
        if piece in ['♖', '♘', '♗', '♕', '♔','♙']:
            return True
        else: return False
        
    def is_black(self,piece):
        if piece in ['♜', '♞', '♝', '♛', '♚','♟']:
            return True
        else: return False
        
    
    def aiplay(self):
        # for all black piece in blackpieces list
        for position in self.blackpos:
            row,col=position[0],position[1]
            piece=self.board[row][col]
            
            # piece,row,col= particular black piece
            
            # Generate possible moves for that black piece
            possible_moves=self.generate_moves(piece,row,col)
            
            # try move one by one
            for taken_move in possible_moves:
                # simulate it and return its initial to taken movement with hvalue
                self.simulate_move(self.board,piece,row,col,taken_move[0],taken_move[1])
            
            # all moves with values
        max_value = max(self.all_plays.values())
        max_keys = [key for key, value in self.all_plays.items() if value == max_value]
        selected_key = random.choice(max_keys)
        return selected_key
            
    
    def generate_moves(self,piece,row,col):
        moves = []
        if piece == '♟':
            # Black Pawn moves
            if row - 1 > -1 and self.board[row - 1][col] == '':
                moves.append([row-1, col])
            if row == 6 and self.board[row-1][col] == '' and self.board[row-2][col] == '':
                moves.append([row - 2, col])
            if row - 1 > -1 and col - 1 >= 0  and self.is_white(self.board[row -1 ][col - 1]):
                moves.append([row - 1, col - 1])
            if row - 1 > -1 and col + 1 < 8 and self.is_white(self.board[row - 1][col + 1]):
                moves.append([row - 1, col + 1])

        elif piece == '♜':
            for direction in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                for i in range(1,8):
                    new_row,new_col=row+direction[0]*i,col+direction[1]*i
                    if 0<=new_row<8 and 0<=new_col<8:
                        if self.board[new_row][new_col]=='':
                            moves.append([new_row,new_col])
                        elif self.is_white(self.board[new_row][new_col]):
                            moves.append([new_row,new_col])
                            break
                        else:
                            break
                
        elif piece == '♞':
            # Black Knight moves
            for drow, dcol in [(2, 1), (2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2), (-2, 1), (-2, -1)]:
                new_row, new_col = row + drow, col + dcol
                if 0 <= new_row < 8 and 0 <= new_col < 8:
                    if self.board[new_row][new_col] == '' or self.is_white(self.board[new_row][new_col]):
                        moves.append([new_row, new_col])

        elif piece == '♝':
            # Black Bishop moves
            for drow, dcol in [(1, 1), (1, -1), (-1, 1), (-1, -1)]:
                for i in range(1, 8):
                    new_row, new_col = row + i * drow, col + i * dcol
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if self.board[new_row][new_col] == '':
                            moves.append([new_row, new_col])
                        elif self.is_white(self.board[new_row][new_col]):
                            moves.append([new_row, new_col])
                            break
                        else:
                            break

        elif piece == '♛':
            for dr in range(-1, 2):
                for dc in range(-1, 2):
                    if dr == 0 and dc == 0:
                        continue
                    for step in range(1, 8):
                        new_row, new_col = row + dr * step, col + dc * step
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if self.board[new_row][new_col] == '':
                                moves.append([new_row, new_col])
                            elif self.is_white(self.board[new_row][new_col]):
                                moves.append([new_row, new_col])
                                break
                            else:
                                break

            # Bishop-like moves (diagonals)
            for dr in range(-1, 2, 2):
                for dc in range(-1, 2, 2):
                    for step in range(1, 8):
                        new_row, new_col = row + dr * step, col + dc * step
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if self.board[new_row][new_col] == '':
                                moves.append([new_row, new_col])
                            elif self.is_white(self.board[new_row][new_col]):
                                moves.append([new_row, new_col])
                                break
                            else:
                                break

        elif piece == '♚':
            # Black King moves
            for drow in range(-1, 2):
                for dcol in range(-1, 2):
                    new_row, new_col = row + drow, col + dcol
                    if 0 <= new_row < 8 and 0 <= new_col < 8 and not (drow == 0 and dcol == 0):
                        if self.board[new_row][new_col] == '' or self.is_white(self.board[new_row][new_col]):
                            moves.append([new_row, new_col])
        return moves
        
    
    def simulate_move(self,board,piece,cur_row,cur_col,selected_row,selected_col):
        # black piece, initial pos to new pos
        new_board = [['' for _ in range(8)] for _ in range(8)]
        # new board for simulating
        for i in range(8):
            for j in range(8):
                new_board[i][j]=board[i][j] 
        
        # captured piece
        captured_piece=new_board[selected_row][selected_col]
        new_board[selected_row][selected_col]=piece
        new_board[cur_row][cur_col]=''
        
        #print(new_board, f'Step taken : from {cur_row},{cur_col}, to {selected_row},{selected_col}')
        hvalue=self.calculate_heuristic(new_board,piece,selected_row,selected_col,captured_piece)
        self.all_plays[(cur_row,cur_col,selected_row,selected_col)]=hvalue
        
    def calculate_heuristic(self,new_board,piece,selected_row,selected_col,captured_piece):
        # black piece under experiment, taken =selected
        maxx_value=0
        
        def whites(new_board):
            whitepos=[]
            for i in range(8):
                for j in range(8):
                    if self.is_white(new_board[i][j])==True:
                        whitepos.append([i,j])
            return whitepos
        #print("Captured piece:",captured_piece)
        if captured_piece in ['♕', '♔']:
            maxx_value+= 3
        elif captured_piece in ['♖', '♘', '♗']:
            maxx_value+= 2
        elif captured_piece == '♙':
            maxx_value+= 1
        elif captured_piece=='':
            pass 
        
        # all pieces if attack -1
        whitepos=whites(new_board)
        for white_pieces in whitepos:
            white_piece = new_board[white_pieces[0]][white_pieces[1]]
            if [selected_row,selected_col] in self.generate_white_moves(new_board,white_piece,white_pieces[0],white_pieces[1]):
                maxx_value -=1
                break
        return maxx_value 
                   
    def generate_white_moves(self,new_board,piece,row,col):
        moves = []
        if piece == '♙':
            # white Pawn moves
            if row + 1 < 8 and new_board[row + 1][col] == '':
                moves.append([row+1, col])
            if row == 1 and new_board[row+1][col] == '' and new_board[row+2][col] == '':
                moves.append([row + 2, col])
            if row + 1 < 8 and col - 1 >= 0  and self.is_black(new_board[row + 1 ][col - 1]):
                moves.append([row + 1, col - 1])
            if row + 1 > -1 and col + 1 < 8 and self.is_black(new_board[row + 1][col + 1]):
                moves.append([row + 1, col + 1])

        elif piece == '♖':
            for direction in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                for i in range(1,8):
                    new_row,new_col=row+direction[0]*i,col+direction[1]*i
                    if 0<=new_row<8 and 0<=new_col<8:
                        if new_board[new_row][new_col]=='':
                            moves.append([new_row,new_col])
                        elif self.is_black(new_board[new_row][new_col]):
                            moves.append([new_row,new_col])
                            break
                        else:
                            break

                
        elif piece == '♘':
            # Black Knight moves
            for drow, dcol in [(2, 1), (2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2), (-2, 1), (-2, -1)]:
                new_row, new_col = row + drow, col + dcol
                if 0 <= new_row < 8 and 0 <= new_col < 8:
                    if new_board[new_row][new_col] == '' or self.is_black(new_board[new_row][new_col]):
                        moves.append([new_row, new_col])

        elif piece == '♗':
            # Black Bishop moves
            for drow, dcol in [(1, 1), (1, -1), (-1, 1), (-1, -1)]:
                for i in range(1, 8):
                    new_row, new_col = row + i * drow, col + i * dcol
                    if 0 <= new_row < 8 and 0 <= new_col < 8:
                        if new_board[new_row][new_col] == '':
                            moves.append([new_row, new_col])
                        elif self.is_black(new_board[new_row][new_col]):
                            moves.append([new_row, new_col])
                            break
                        else:
                            break

        elif piece == '♕':
            for dr in range(-1, 2):
                for dc in range(-1, 2):
                    if dr == 0 and dc == 0:
                        continue
                    for step in range(1, 8):
                        new_row, new_col = row + dr * step, col + dc * step
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if new_board[new_row][new_col] == '':
                                moves.append([new_row, new_col])
                            elif self.is_black(new_board[new_row][new_col]):
                                moves.append([new_row, new_col])
                                break
                            else:
                                break

            # Bishop-like moves (diagonals)
            for dr in range(-1, 2, 2):
                for dc in range(-1, 2, 2):
                    for step in range(1, 8):
                        new_row, new_col = row + dr * step, col + dc * step
                        if 0 <= new_row < 8 and 0 <= new_col < 8:
                            if new_board[new_row][new_col] == '':
                                moves.append([new_row, new_col])
                            elif self.is_black(new_board[new_row][new_col]):
                                moves.append([new_row, new_col])
                                break
                            else:
                                break

        elif piece == '♔':
            # Black King moves
            for drow in range(-1, 2):
                for dcol in range(-1, 2):
                    new_row, new_col = row + drow, col + dcol
                    if 0 <= new_row < 8 and 0 <= new_col < 8 and not (drow == 0 and dcol == 0):
                        if new_board[new_row][new_col] == '' or self.is_black(new_board[new_row][new_col]):
                            moves.append([new_row, new_col])
        return moves
                
class ChessBoard:
    def __init__(self, master):
        self.master = master
        self.square_size = 75
        self.master.title("Chess Board GUI")
        self.board = [['' for _ in range(8)] for _ in range(8)]
        self.rectangle_ids = [['' for _ in range(8)] for _ in range(8)]
        self.text_ids= [['' for _ in range(8)] for _ in range(8)]
        self.canvas = tk.Canvas(self.master, width=600, height=600)
        self.canvas.grid()
        self.draw_board()
        self.selected_piece = None  
        self.selected_rectangle = None
        self.prow,self.pcol=None,None
        self.counter=0
        self.selected_key=(0,0,0,0)
        messagebox.showinfo("Welcome!", "White turns first!")
        
    def blacks(self):
        blackpos=[]
        for i in range(8):
            for j in range(8):
                if self.is_black(self.board[i][j])==True:
                    blackpos.append([i,j])
        return blackpos
    
    def assign_rectangle_id(self,row,col,rectangle_id):
        self.rectangle_ids[row][col]=rectangle_id
    
    def assign_text_id(self,row,col,text_id):
        self.text_ids[row][col]=text_id
        
    def assign_board(self,row,col,piece):
        self.board[row][col]=piece
    
    def remove_text_id(self,row,col,text_id):
        self.canvas.delete(text_id)
        
    def display_piece(self, row, col):
        text_id = self.text_ids[row][col]
        piece = self.board[row][col]
        self.canvas.itemconfigure(text_id, text=piece)
        
    def is_white(self,piece):
        if piece in ['♖', '♘', '♗', '♕', '♔','♙']:
            return True
        else: return False
        
    def is_black(self,piece):
        if piece in ['♜', '♞', '♝', '♛', '♚','♟']:
            return True
        else: return False
    
    def is_king_under_attack(self, king_piece):
        # Find the coordinates of the king
        king_row, king_col = None, None
        for row in range(8):
            for col in range(8):
                if self.board[row][col] == king_piece:
                    king_row, king_col = row, col
                    break
            if king_row is not None:
                break
        rectangle_id=self.rectangle_ids[king_row][king_col]
        # Check if the king is under attack by any opposing pieces
        for row in range(8):
            for col in range(8):
                piece = self.board[row][col]
                if piece != '' and ((self.is_black(king_piece) and self.is_white(piece)) or (self.is_white(king_piece) and self.is_black(piece))):
                    if self.move(piece, row, col, king_row, king_col):
                        self.canvas.itemconfig(rectangle_id, outline='#e03009', width=2)
                        return True
                
        return False
    
    def move(self, piece, start_row, start_col, dest_row, dest_col):
        # Get the direction of movement (horizontal, vertical, diagonal)
        delta_row = dest_row - start_row
        delta_col = dest_col - start_col
        #print( piece, start_row, start_col, dest_row, dest_col,delta_row,delta_col)

        # Check if the destination is within the board boundaries
        if dest_row < 0 or dest_row >= 8 or dest_col < 0 or dest_col >= 8:
            return False

        # Check if the destination is occupied by a piece of the same color
        if self.is_white(piece) and self.is_white(self.board[dest_row][dest_col]):
            return False
        if self.is_black(piece) and self.is_black(self.board[dest_row][dest_col]):
            return False

        # Handle the movement based on the piece type
        if piece in ['♖', '♜']:  # Rook
            if delta_row != 0 and delta_col != 0:
                return False  # Rook can only move horizontally or vertically

            # Check for obstacles along the path
            if delta_row == 0:  # Horizontal movement
                start, end = min(start_col, dest_col), max(start_col, dest_col)
                for col in range(start + 1, end):
                    if self.board[start_row][col] != '':
                        return False
                    
            else:  # Vertical movement
                start, end = min(start_row, dest_row), max(start_row, dest_row)
                for row in range(start + 1, end):
                    if self.board[row][start_col] != '':
                        return False
            return True

        elif piece in ['♗', '♝']:  # Bishop
            if abs(delta_row) != abs(delta_col):
                return False  # Bishop can only move diagonally

            # Check for obstacles along the path
            row_step = 1 if delta_row > 0 else -1
            col_step = 1 if delta_col > 0 else -1
            row, col = start_row + row_step, start_col + col_step
            while row != dest_row:
                if self.board[row][col] != '':
                    return False
                row += row_step
                col += col_step
            return True

        elif piece in ['♕', '♛']:  # Queen        
            if abs(delta_row) == abs(delta_col):
                # Check for obstacles along the diagonal path
                row_step = 1 if delta_row > 0 else -1
                col_step = 1 if delta_col > 0 else -1
                row, col = start_row + row_step, start_col + col_step
                while row != dest_row:
                    if self.board[row][col] != '':
                        return False
                    row += row_step
                    col += col_step
                return True
            
            elif delta_row == 0 or delta_col == 0:
                # Check for obstacles along the horizontal or vertical path
                if delta_row == 0:  # Horizontal movement
                    start, end = min(start_col, dest_col), max(start_col, dest_col)
                    for col in range(start + 1, end):
                        if self.board[start_row][col] != '':
                            return False
                else:  # Vertical movement
                    start, end = min(start_row, dest_row), max(start_row, dest_row)
                    for row in range(start+1, end):
                        if self.board[row][start_col] != '':
                            return False
                return True
            else:
                return False  # Invalid movement for the queen

        elif piece in ['♘', '♞']:  # Knight
            # Knights move in an L-shape pattern
            if abs(delta_row) == 2 and abs(delta_col) == 1:
                # Valid vertical L-shape movement
                return True
            elif abs(delta_row) == 1 and abs(delta_col) == 2:
                # Valid horizontal L-shape movement
                return True
            else:
                return False

        elif piece in ['♔', '♚']:  # King
            if abs(delta_row) <= 1 and abs(delta_col) <= 1:
                # Valid king movement (up to 1 square in any direction)
                return True
            else:
                return False

        elif piece == '♙':  # White Pawn
            if delta_col == 0 and delta_row == 1 and self.board[dest_row][dest_col] == '':
                # Pawn moves one step forward
                return True
            elif delta_col == 0 and delta_row == 2 and start_row == 1 and self.board[dest_row][dest_col] == '':
                # Pawn moves two steps forward from the initial position
                return True
            elif abs(delta_col) == 1 and delta_row == 1 and self.is_black(self.board[dest_row][dest_col]):
                # Pawn captures diagonally
                return True
            else:
                return False

        elif piece == '♟':  # Black Pawn
            if delta_col == 0 and delta_row == -1 and self.board[dest_row][dest_col] == '':
                # Pawn moves one step forward
                return True
            elif delta_col == 0 and delta_row == -2 and start_row == 6 and self.board[dest_row][dest_col] == '':
                # Pawn moves two steps forward from the initial position
                return True
            elif abs(delta_col) == 1 and delta_row == -1 and self.is_white(self.board[dest_row][dest_col]):
                # Pawn captures diagonally
                return True
            else:
                return False

        return False  # Invalid move
        
    def draw_board(self):
        for row in range(8):
            for col in range(8):
                x1 = col * self.square_size 
                y1 = row * self.square_size 
                x2 = x1 + self.square_size 
                y2 = y1 + self.square_size 
                
                if (row + col) % 2 == 0:
                    rectangle_id=self.canvas.create_rectangle(x1, y1, x2, y2, fill='#4481A7',width=2)
                else:
                    rectangle_id=self.canvas.create_rectangle(x1, y1, x2, y2, fill='white',width=2)
                    
                # Assign rectangle id to rectangle ids matrix
                self.assign_rectangle_id(row,col,rectangle_id)
                
                # Assign text_id to text_ids8
                text_id=self.canvas.create_text(x1 + self.square_size/2, y1 + self.square_size/2, text=self.board[row][col], font=("Arial", 20))
                self.assign_text_id(row,col,text_id)
                
                self.canvas.tag_bind(rectangle_id, '<Button-1>', self.rectangle_clicked)       
        self.place_piece('♔', 0, 4)
        self.place_piece('♚', 7, 4)
        self.place_piece('♕', 0, 3)
        self.place_piece('♛', 7, 3)
        self.place_piece('♖', 0, 0)
        self.place_piece('♜', 7, 0)
        self.place_piece('♗', 0, 2)
        self.place_piece('♝', 7, 2)
        self.place_piece('♘', 0, 1)
        self.place_piece('♞', 7, 1)
        self.place_piece('♗', 0, 5)
        self.place_piece('♝', 7, 5)
        self.place_piece('♘', 0, 6)
        self.place_piece('♞', 7, 6)
        self.place_piece('♖', 0, 7)
        self.place_piece('♜', 7, 7)
        for i in range(8):
            self.place_piece('♙', 1, i)
            self.place_piece('♟', 6, i)
        
    def rectangle_clicked(self, event):
        rectangle_id = event.widget.find_closest(event.x, event.y)[0]
        row, col = self.find_rectangle_coordinates(rectangle_id)
        clicked_piece = self.board[row][col]

        #  piece selected for 1st time
        if self.selected_piece is None:
            # if white or black is selected else nothing
            if clicked_piece != '' and ((self.counter%2!=0 and self.is_black(clicked_piece)) or (self.counter%2==0 and self.is_white(clicked_piece))):
                self.selected_piece = clicked_piece
                self.selected_rectangle = rectangle_id
                self.prow,self.pcol=row,col
                self.canvas.itemconfig(rectangle_id, outline='#7AFFF6', width=2)
                
        # selected piece is selected again      
        elif self.selected_rectangle == rectangle_id:
            # Deselect the previously selected piece
            self.selected_piece = None
            self.prow, self.pcol = None, None
            self.canvas.itemconfig(rectangle_id, outline='', width=2)
                
        # attack        
        else:
            # same color attack
            if (self.is_black(self.selected_piece) and self.is_black(clicked_piece)) or (self.is_white(self.selected_piece) and self.is_white(clicked_piece)):
                pass
            # attacked
            else:
                if self.move(self.selected_piece,self.prow,self.pcol,row,col) is True:
                    selected_row, selected_col = self.find_rectangle_coordinates(self.selected_rectangle)
                    self.place_piece('', selected_row, selected_col)  # Clear the previous position
                    self.place_piece(self.selected_piece, row, col)  # Move the piece to the new position
                    self.selected_piece = None
                    self.prow,self.pcol=None,None
                    self.canvas.itemconfig(self.selected_rectangle, outline='', width=2)
                    
                    if self.is_king_under_attack('♔'):
                        messagebox.showinfo("Danger!", "White King in danger!")
                    elif self.is_king_under_attack('♚'):
                        messagebox.showinfo("Danger!", "Black King in danger!")
                        
                    self.counter+=1
                    print(self.counter)
                    
                    if self.counter%2!=0:
                        blackpos=self.blacks()
                        next_move=AI(self.board,blackpos)
                        self.selected_key=next_move.aiplay()
                        print(self.selected_key)
                        self.AIMOVE(self.selected_key[0],self.selected_key[1])
                        self.AIMOVE(self.selected_key[2],self.selected_key[3])
                else:
                    print('Invalid move')
                    
    def AIMOVE(self,row,col):
        rectangle_id=self.rectangle_ids[row][col]
        clicked_piece = self.board[row][col]

        #  piece selected for 1st time
        if self.selected_piece is None:
            # if white or black is selected else nothing
            if clicked_piece != '' and ((self.counter%2!=0 and self.is_black(clicked_piece)) or (self.counter%2==0 and self.is_white(clicked_piece))):
                self.selected_piece = clicked_piece
                self.selected_rectangle = rectangle_id
                self.prow,self.pcol=row,col
                self.canvas.itemconfig(rectangle_id, outline='#7AFFF6', width=2)
                
        # selected piece is selected again      
        elif self.selected_rectangle == rectangle_id:
            # Deselect the previously selected piece
            self.selected_piece = None
            self.prow, self.pcol = None, None
            self.canvas.itemconfig(rectangle_id, outline='', width=2)
                
        # attack        
        else:
            # same color attack
            if (self.is_black(self.selected_piece) and self.is_black(clicked_piece)) or (self.is_white(self.selected_piece) and self.is_white(clicked_piece)):
                pass
            # attacked
            else:
                if self.move(self.selected_piece,self.prow,self.pcol,row,col) is True:
                    selected_row, selected_col = self.find_rectangle_coordinates(self.selected_rectangle)
                    self.place_piece('', selected_row, selected_col)  # Clear the previous position
                    self.place_piece(self.selected_piece, row, col)  # Move the piece to the new position
                    self.selected_piece = None
                    self.prow,self.pcol=None,None
                    self.canvas.itemconfig(self.selected_rectangle, outline='', width=2)
                    if self.is_king_under_attack('♔'):
                        messagebox.showinfo("Danger!", "White King in danger!")
                    elif self.is_king_under_attack('♚'):
                        messagebox.showinfo("Danger!", "Black King in danger!")
                    self.counter+=1
                    print(self.counter)
                else:
                    print('Invalid move')
                

    def find_rectangle_coordinates(self, rectangle_id):
        for row in range(8):
            for col in range(8):
                if self.rectangle_ids[row][col] == rectangle_id:
                    return row, col
            
    def place_piece(self, piece, row, col):
        x1 = col * self.square_size
        y1 = row * self.square_size
        self.assign_board(row, col, piece)  # Update the board first
        text_id = self.text_ids[row][col]
        self.canvas.itemconfigure(text_id, text=piece,width=2)  # Update the text of the existing text item
        
    def run(self):
        self.master.mainloop()
        
root = tk.Tk()
chessboard = ChessBoard(root)
chessboard.run()

1
(6, 5, 5, 5)
2
3
(6, 1, 5, 1)
4
5
(6, 6, 4, 6)
6
7
(4, 6, 3, 5)
8
9
(6, 3, 5, 3)
10
11
(7, 2, 3, 6)
12
13
(3, 6, 2, 5)
14
Invalid move
15
(7, 5, 5, 7)
16
